In [ ]:
"""Module import"""

from collections import OrderedDict, defaultdict

"""Import module of openalea packages"""
from openalea.core import path
from openalea.mtg import *
from openalea.mtg.algo import orders
from openalea.mtg import stat
from openalea.deploy.shared_data import shared_data
import openalea.plantgl.all as pgl

""" Import module of strawberry packages"""
import openalea.strawberry
from openalea.strawberry import Rules_production
from openalea.strawberry import Visualization
from openalea.strawberry import analysis

In [ ]:
% gui qt
files = shared_data(openalea.strawberry).glob('*.mtg')
files = files[0:-1]
files

In [ ]:
""" Merge all MTG to realize au big_MTG with all varieties"""
big_mtg = MTG()
for fn in files:
    big_mtg = algo.union(big_mtg, MTG(fn))
g = big_mtg

genotypes = set(g.property("Genotype").itervalues())
print genotypes

In [ ]:
from strawberry import variables

In [ ]:
g=MTG(files[0])
files[0]


In [ ]:
g.property_names()

# Sequence extraction according to Modules 
## For each module (scale = 2) qui porte un F qui se termine par HT, ht, bt


### Index
- real Order (Index) for recouvrant extraction
- Apparent Order for non recouvrant extraction

### Variables:
 - nb_visible_leaves : count F label in components
 - nb_foliar_primordia: count f si le module commence par F
 - nb_total_leaves: sum(F+f) du module
 - Stolon Numbers: count s label in components and attach at the order -1 
 - nb_open_flowers: g.property('Fleurs_ouverte') at scale 3
 - nb_aborted_flowers: g.property('Fleurs_avorte') at scale 3
 - nb_total_flowers : g.property('Fleurs_total') at scale 3
 - No_vegetative_bud: count bt if g.property(Stade) == none or 17 or 18 or 19
 - No_iniated_bud: count bt if g.property(Stade) == A
 - No_Floral_bud: count ht
 - No_Inflorescence: count HT
 
###Variable qualitative
 - type of crown
     - label == T (1) : principal crown
     - parent(component_roots()[0]) : if successor() == F : branch_crown elif in (HT, ht, bt) extension crown
 - inflorescence_type:
     - vegetative (1): label==bt g.property(Stade)== none or 17 or 18 or 19
     - initiated (2): label== bt if g.property(Stade) == A
     - floral (3): label== ht or HT

### General Variables
- Genotype: g.property('Genotype') at scale 1
- Date: g.property('date') at scale 1
- Plante Number: g.property('plante') at scale 1

In [ ]:
# apparent_module: all module which contains a visible leaf F
def visible_modules(g=g):
    return [v for v in g.vertices_iter(scale=2) if g.label(g.component_roots_iter(v).next()) == 'F']

In [ ]:
# add property in MTG, apparant module (True) or not
modules = visible_modules()
_visible = {}
for m in modules:
    _visible[m] = True
g.properties()['visible'] = _visible


In [ ]:
"""Index

TODO: Replace dict by functions with specific traversal.

"""
#order scale 2: order at module scale
orders = algo.orders(g, scale=2)

"""Variable quantitative"""
#function which count all F
def nb_visible_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='F')

#function which count all f
def nb_foliar_primordia(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='f')

#function which count all f+F
def nb_total_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid) in ('f', 'F'))

""" nb_stolon"""
#function count stolon
def nb_stolons(v, g=g):
    def nb_stolon(vid, g=g):
        return sum(1 for cid in g.components(vid) if g.label(cid)=='s') 
    return sum(nb_stolon(ch) for ch in g.children(v))

#function return number of open flower
def nb_open_flowers(vid, g=g):
    flowers = g.property('Fleurs_ouverte')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

# function return number of aborted flower
def nb_aborted_flowers(vid, g=g):
    flowers = g.property('Fleurs_aborted')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

# function return number of total flower
def nb_total_flowers(vid, g=g):
    flowers = g.property('Fleurs_total')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))


"""Return the No vegetative bud
    
Algorithm:
if labet is bt then stage is 17,18,19 or None
count number of bt and attach at the parent order
    """
# function return number of vegetative buds
def nb_vegetative_buds(vid, g=g):
    stages= g.property('Stade')

    def nb_vegetative(v):
        cid = g.component_roots(v)[0]
        
        return sum(1 for cid in g.components(v) if g.label(cid)=='bt' and stages.get(cid) in (None,'17','18','19'))
    
    return sum(nb_vegetative(ch) for ch in g.children(vid))

 
""" Return the No initiated bud"""

def nb_initiated_buds(vid, g=g):
    
    stages= g.property('Stade')
    
    def nb_init(v):
        return sum(1 for cid in g.components(v) if (g.label(cid)=='bt') and (stages.get(cid)=='A'))

    return sum(nb_init(ch) for ch in g.children(vid))


""" Return the No Floral bud"""
def nb_floral_buds (vid, g=g):
    visibles = g.property('visible')
    def nb_floral(v):
        return sum(1 for cid in g.components(v) if g.label(cid)=="ht" )
    return sum(nb_floral(ch) for ch in g.children(vid) if ch not in visibles)


""" Qualitative variables"""
def type_of_crown(vid, g=g):
    """ Returns the type of crown.
    
    Definition of type of crown (1, 2, 3):
     - principal crown (1): label == T 
     - branch_crown (3)
         parent(component_roots()[0]) : if successor() == F  
     - extension_crown (2): contains(HT, ht, bt) 
     - error (4)
     
    """
    if g.label(vid) == 'T':
        return 1
    else:
        cid = g.component_roots_iter(vid).next()
        pid = g.parent(cid)
        sid = g.Successor(pid)
        #print sid
        if g.label(sid) in ('F', 'f'):
            return 3
        elif g.label(sid) in ('bt', 'ht', 'HT'):
            return 2
        else:
            # ERROR !!!
            print g[cid], g[g.complex_at_scale(cid, scale=1)]
            return 4
            
def Crown_status(vid, g=g):
    """ Returns the type of inflorescence
    
    :Algorithms:
    if label is bt then
        - if stage is 17, 18, 19 or None, => vegetative (1)
        - if stage is A => initiated (2)
        - if stage is other => non defined (pourri, avorté, coupé) (-1)
     - Terminal vegetative bud (1): label==bt g.property(Stade)== none or 17 or 18 or 19
     - Terminal initiated bud (2): label== bt if g.property(Stade) == A
     - Terminal Floral bud (3): label==ht
     - Inflorescence Terminal (4): label== HT
     - runner (5): label = s
     
    """
    stages = g.property('Stade')
    # select s, ht, HT et bt
    for cid in g.components(vid):
        if g.label(cid) in ('s', 'ht', 'HT', 'bt'):
            label = g.label(cid)
            if label == 'ht':
                return 3
            elif label == 'HT':
                return 4
            elif label == 'bt':
                stage = stages.get(cid)
                if stage == 'A':
                    return 2
                elif stage in (None, '17', '18', '19'):
                    return 1
            elif label == 's':
                return 5
            return -1

def nb_inflorescence (Vid, g):
    return sum(1 for cid in g.components(Vid) if g.label(cid)=='HT')
    
def genotype(vid, g=g):
    d = {'Capriss':4, 'Ciflorette':2, 'Cir107':6, 'Clery':3, 'Darselect':5, 'Gariguette':1}
    
    cpx = g.complex_at_scale(vid, scale=1)
    _genotype = g.property('Genotype')[cpx]
    return d[_genotype]

def plant(vid, g=g):
    
    cpx = g.complex_at_scale(vid, scale=1)
    return g.property('Plante')[cpx]

def date(vid, g=g):
    d = {'10-12-2014':1,'07-01-2015':2,'15-02-2015':3,'02-03-2015':4,'03-04-2015':5,'27-05-2015':6,'04-12-2014':1,'13-02-2015':3,
         '30-03-2015':5,'08-01-2015':2,'11-02-2015':3,'04-03-2015':4,'02-04-2015':5,'20-05-2015':6,'09-01-2015':2,'06-03-2015':4,
         '12-02-2015':3,'19-05-2015':6}         
    cpx = g.complex_at_scale(vid, scale=1)
    _date = g.property('date')[cpx]
    return(d[_date])

In [ ]:
# Extract sequences

#seqs = stat.extract_extremities(g, scale=2)
#seqs = stat.extract_axes(g, scale=2)
def is_extremities(g, v):
    visibles = g.property('visible')
    ext = [c for c in g.children(v) if c in visibles and not is_axis_root(g, c)]
    return True if not ext else False

def extract_extremities(g, modules):
    """ Extract sequences that connect from leaves modules to the root ones of each plant. """
    leaves = [m for m in modules if is_extremities(g, m)]
    seqs = [list(reversed([vid for vid in algo.ancestors(g,lid)])) for lid in leaves]
    return seqs

def is_axis_root(g, vid):
    cid = g.component_roots_iter(vid).next()
    pid = g.parent(cid)
    sid = g.Successor(pid)
    if g.label(sid) not in ('bt', 'ht', 'HT'):
        return True
    else:
        return False
    

def apparent_axis(g, vid):
    visibles = g.property('visible')
    v = vid
    while v is not None:
        yield v
        vtx = v; v = None
        for vid in g.children(vtx):
            if (vid in visibles) and (not is_axis_root(g, vid)):
                v = vid
            
def extract_apparent_axes(g, modules):
    # Extract all the vertices with edge_type == '+'
    vids = [m for m in modules if is_axis_root(g, m)]
    
    seqs = [ list(apparent_axis(g, vid)) for vid in vids]
    return seqs


In [ ]:
seqs1 = extract_apparent_axes(g, modules)
seqs2= extract_extremities(g,modules)
seqs = seqs1
len (seqs)

In [ ]:

# Index: order
VertexIdentifiers = [['-'.join(str(orders[j]) for j in seq[0:i+1]) for i, vid in enumerate(seq)] for seq in seqs]

# Variables
variables = OrderedDict()
variables['nb_visible_leaves'] = nb_visible_leaves # Nombre de feuille developpe
variables['nb_foliar_primordia'] = nb_foliar_primordia #Nombre de primordia foliaire
variables['nb_total_leaves'] = nb_total_leaves #Nombre total de feuille
variables['nb_open_flowers'] = nb_open_flowers #Nombre de fleurs ouverte
variables['nb_aborted_flowers'] = nb_aborted_flowers #Nombre de fleurs avorté
variables['nb_total_flowers'] = nb_total_flowers #Nombre total de Fleurs
variables['vegetative_bud'] = nb_vegetative_buds
variables['Initiated_bud']= nb_initiated_buds
variables['floral_bud']= nb_floral_buds
variables['stolons']= nb_stolons
variables['type_of_crown'] = type_of_crown # Type de crowns (Primary Crown:1, Branch crown:2 extension crown:3)
variables['Crown_status'] = Crown_status # Status du coeurs ("vegeatatif:1, Iniated: 2, Floral: 3)


variables['genotype'] = genotype # TODO
variables['date'] = date # TODO
variables['plant'] = plant

In [ ]:
variables

In [ ]:
variable_names = variables.keys()
print variable_names

final_seqs = [[[f(vid) for f in variables.values()] for vid in seq] for seq in seqs]

In [ ]:
sequence_txt = analysis.write_sequences(final_seqs, variable_names, VertexIdentifiers)

In [ ]:
g.property_names()

In [ ]:
# Check variables....
print sequence_txt

In [ ]:
f = open('strawberry_archi_module_nonoverlaping_path.seq', 'w') #seq1
#f = open('strawberry_archi_module_overlaping_path.seq', 'w') #seq2

f.write(sequence_txt)
f.close()

##### Test the different variables

In [ ]:
for name, f in variables.iteritems():
    print name
    #print set(f(m) for m in modules)

In [ ]:
test = [m for m in modules if plant(m) == 2 and genotype(m)==4 and date(m)==1]
test

In [ ]:
nb_floral_buds(123)

# Extraction Node scale
## 1.Index
Index is node on apparent trunk axis

## 2. Qualitative variable
On each nodes identify the type of ramification:
ramif-type: 
- if is invisible modules==F take form of: 
    - stolon (s): 1, 
    - vegetative bud(bt, at stage None, 17,18,19):2,
    - initiated bud, (bt, at stage A):3, 
    - floral bud(ht):4 
- if is visible module==T: 
    -branch crown (complex ramification):5

## 3. Variables
if branch crown(5):
- on principal path:
    - No. modules
    - No. developed leaves (F)
    - No. primordia leaves (f)
    - No. total leaves (f+F)
    - No. open flower, g.property("open_flower")
    - No. aborded flower, g.property("aborded_flower")
    - No. total flower, g.property("total flower")
- on whole branch crown:
    - No. branch crown carried sum(No. visible modules if modules is_axis_root==T )
    - No. extension crown carried sum(No. visible module if modules is_axis_root==F) 
    - No. total modules carried 
        - sum(No.visible modules)
    - No. developed leaves carried 
            sum(for vid in g if g.label=="F" and if visible module==T)
    - No. primordia leaves carried sum(f, if visible module==T)
    - No. total leaves carried sum(F+f, if visible module==T)
    - No. vegetative buds carried sum(bt at stage None, 17,18,19 if visible modules ==F)
    - No. initiated buds carried sum(bt at stage A if visible modules == F)
    - No. floral buds carried sum(ht if visible modules==F)
    - No. Inflorescence carried sum(HT if visible module==T)
    - No. open flowers carried sum(g.property("open_flowers"), if visible modules==T)
    - No. aborded flowers carried sum(g.property("aborded_flower"if visible module==T)
    - No. total flowers carried sum(g.property("total_total flowers" if visible module==T)
    - No. stolon carried sum(s, if visible module ==F)
 

In [ ]:
from itertools import chain
from openalea.mtg import algo, traversal

In [ ]:
# Extract the first module of each plant at scale 2
roots = g.component_roots_at_scale(g.root, scale=2)
# assert set(g.label(r) for r in roots) == set(['T']) 

trunks = [ list(chain(*[(v for v in algo.axis(g,m, scale=3) if g.label(v) in ('F', 'f')) for m in apparent_axis(g, r)])) for r in roots]
# print trunk

# define the apparent order on the MTG
vertex_ids = [[str(rank) for rank, v in enumerate(trunk)] for trunk in trunks]

# extract a sequence of F for each module

In [ ]:
def branching(g, v):
    sid = g.Successor(v)
    if g.label(sid) in ('bt', 'ht', 'HT'):
        return None
    
    kids = [c for c in g.children(v) if g.edge_type(c) =='+' and g.label(c) == 'F' ]
    if not kids:
        return None
    
    return kids[0]

def branching_type(vid, g=g):
    """ Returns the type of branching
    
    :Algorithms:
    
    if module is visible:
        - branch crown (complex ramification):6
        - inflorescence : 7

    if module is invisible:
        - stolon (s): 1,
        - vegetative bud(bt, at stage None, 17,18,19):2,
        - initiated bud, (bt, at stage A):3,
        - aborted or roten or dried bud: 4
        - floral bud(ht):5

     
    """

    cpx = g.complex(vid)
    nxid = g.node(cpx) 
    if nxid.visible:
        if g.label(vid) == 'HT':
            return 7
        else:
            return 6
    
    # select s, ht, et bt
    for cid in nxid.components():
        label = cid.label
        if label in ('s', 'ht', 'bt'):
            if label == 's':
                return 1
            elif label == 'bt':
                stage = cid.Stade
                if stage in (None, '17', '18', '19'):
                    return 2
                elif stage == 'A':
                    return 3
                elif stage in ('pourri', 'aborted', 'dried'):
                    return 4
            elif label  == 'ht':
                return 5
    else:
        print 'ERROR: ', cpx, nxid.complex().Genotype, nxid.properties()

#[[branching(g, v) for v in trunk] for trunk in trunks]

# Extract on a vertex the tree of modules
def module_tree(v, g):
    visibles = g.property('visible')
    cpx = g.complex(v)
    return [m for m in traversal.pre_order2(g, cpx) if m in visibles]


In [ ]:
def module_axis(v, g=g):
    cpx = g.complex(v)
    return list(apparent_axis(g, cpx))

def nb_modules_axis(v, g=g):
    return len(module_axis(v, g=g))

def nb_visible_leaves_axis(v, g=g):
    return sum(nb_visible_leaves(m, g) for m in module_axis(v, g=g))

def nb_foliar_primordia_axis(v, g=g):
    return sum(nb_foliar_primordia(m, g) for m in module_axis(v, g=g))

def nb_total_leaves_axis(v, g=g):
    return sum(nb_total_leaves(m, g) for m in module_axis(v, g=g))

def nb_open_flowers_axis(v, g=g):
    return sum(nb_open_flowers(m, g) for m in module_axis(v, g=g))

def nb_aborted_flowers_axis(v, g=g):
    return sum(nb_aborted_flowers(m, g) for m in module_axis(v, g=g))

def nb_total_flowers_axis(v, g=g):
    return sum(nb_total_flowers(m, g) for m in module_axis(v, g=g))

def nb_vegetative_buds_axis(v, g=g):
    return sum(nb_vegetative_buds(m, g) for m in module_axis(v, g=g))

def nb_initiated_buds_axis(v, g=g):
    return sum(nb_initiated_buds(m, g) for m in module_axis(v, g=g))

def nb_floral_buds_axis(v, g=g):
    return sum(nb_floral_buds(m, g) for m in module_axis(v, g=g))

def nb_stolons_axis(v, g=g):
    return sum(nb_stolons(m, g) for m in module_axis(v, g=g))


####################################################
def nb_total_module_tree(v, g=g):
    return len(module_tree(v, g))

def nb_branching_tree(v, g=g):
    return sum(1 for m in module_tree(v, g) if is_axis_root(g, m))

def nb_extension_tree(v,g=g):
    return sum(1 for m in module_tree(v,g) if not is_axis_root(g, m))

def nb_visible_leaves_tree(v, g=g):
    return sum(nb_visible_leaves(m,g) for m in module_tree(v, g))

def nb_primordia_leaves_trees(v, g=g):
    return sum(nb_foliar_primordia(m,g) for m in module_tree(v, g))

def nb_total_leaves_trees(v, g=g):
    return sum(nb_total_leaves(m, g) for m in module_tree(v, g))

def nb_vegetative_buds_tree(v, g=g):
    return sum(nb_vegetative_buds(m, g) for m in module_tree(v, g))

def nb_initiated_buds_tree(v, g=g):
    return sum(nb_initiated_buds(m, g) for m in module_tree(v, g))

def nb_floral_buds_tree(v, g=g):
    return sum(nb_floral_buds(m, g) for m in module_tree(v, g))

def nb_inflorescences_tree(v, g=g):
    return sum(nb_inflorescences(m, g) for m in module_tree(v, g))

def nb_open_flowers_tree(v, g=g):
    return sum(nb_open_flowers(m, g) for m in module_tree(v, g))

def nb_aborted_flowers_tree(v , g=g):
    return sum(nb_aborted_flowers(m, g=g) for m in module_tree(v, g))

def nb_total_flowers_tree(v, g=g):
    return sum(nb_total_flowers(m, g=g) for m in module_tree(v, g))

def nb_stolon_tree(v, g=g):
    return sum(nb_stolons(m, g) for m in module_tree(v, g))

def nb_inflorescences_tree(v, g=g):
    return sum(nb_inflorescence (m, g) for m in module_tree(v, g))

##################################################
def genotype_axis(v, g=g):
    return genotype(g.complex(v), g)
def date_axis(v, g=g):
    return date(g.complex(v), g)
def plant_axis(v, g=g):
    return plant(g.complex(v), g)
# test
#trunk = trunks[1]
#[nb_stolon_tree(branching(g, v)) for v in trunk if branching(g, v)]


In [ ]:
# Variables
variables = OrderedDict()

# on principal ramification path
#variables["branching_type"]= branching_type
variables['nb_module_axis'] = nb_modules_axis
variables['nb_visible_leaves_axis'] = nb_visible_leaves_axis # Nombre de feuille developpe
variables['nb_foliar_primordia_axis'] = nb_foliar_primordia_axis #Nombre de primordia foliaire
variables['nb_total_leaves_axis'] = nb_total_leaves_axis #Nombre total de feuille
variables['nb_vegetative_buds_axis']= nb_vegetative_buds_axis #
variables['nb_initiated_buds_axis']= nb_initiated_buds_axis
variables['nb_floral_buds_axis']= nb_floral_buds_axis
variables['nb_open_flowers_axis'] = nb_open_flowers_axis #Nombre de fleurs ouverte
variables['nb_aborted_flowers_axis'] = nb_aborted_flowers_axis #Nombre de fleurs avorté
variables['nb_total_flowers_axis'] = nb_total_flowers_axis #Nombre total de Fleurs
variables['nb_stolons_axis'] = nb_stolons_axis

#on whole ramification 
variables["nb_branch_crown_tree"]= nb_branching_tree # sum(No. visible modules if modules is_axis_root )
variables["nb_extenstion_crown_tree"]= nb_extension_tree # sum(No. visible module if modules nit is_axis_root)
variables["nb_total module_tree"]= nb_total_module_tree # sum(visible modules)
variables["nb_visible_leaves_tree"]= nb_visible_leaves_tree # sum(F, if visible module==T)
variables["nb_primordia_leaves_trees"]=nb_primordia_leaves_trees #sum(f, if visible module==T)
variables["nb_total_leaves_trees"]=nb_total_leaves_trees #sum(F & f , if visible module==T)
variables["nb_vegetative_buds_tree"]= nb_vegetative_buds_tree# sum(bt if stages in (None,'17','18','19') and visible module==F)
variables["nb_initiated_buds_tree"]= nb_initiated_buds_tree# sum(bt if stages=='A' and visible module==F)
variables["nb_floral_buds_tree"]= nb_floral_buds_tree# sum(ht if visible module==F)
variables["nb_inflorescence_tree"]= nb_inflorescences_tree# sum(HT if visible module==T)
variables["nb_open_flowers_tree"]= nb_open_flowers_tree# sum(g.property('Fleurs_ouverte') if visible module==T)
variables["nb_aborted_flowers_tree"]= nb_aborted_flowers_tree# sum(g.property('Fleurs_aborted') if visible module==T)
variables["nb_total_flowers_tree"]= nb_total_flowers_tree# sum(g.property('Fleurs_total') if visible module==T)
variables["nb_stolon_tree"]= nb_stolon_tree# sum(s if visible modules ==F)

# add supplemental variables
#variables No. Inflorescence
#variables No. Inflorescence crown bearing by ramification
#variables No Vegetative crown bearing by ramification
#variables sum initiated crown bearing by ramification
#variable sum floral crown bearing by ramification

#Global variable
global_variables = OrderedDict()
global_variables['genotype'] = genotype_axis # TODO
global_variables['date'] = date_axis # TODO
global_variables['plant'] = plant_axis

# Extract variables fo Inflorescences
inflo_variables = OrderedDict()
inflo_variables['nb_total_flowers_axis'] = nb_total_flowers 
inflo_variables['nb_open_flowers_axis'] = nb_open_flowers #Nombre de fleurs ouverte
inflo_variables['nb_aborted_flowers_axis'] = nb_aborted_flowers #Nombre de fleurs avorté



In [ ]:
final_seqs = []
# add if not ramif:
def is_inflo(g, vid):
    return True if g.label(vid) =='HT' else False

def filter_hamp_branch_crown(g, vid):
    """Extract first ramification node or inflo. """
    inflo = [c for c in g.children(vid) if is_inflo(g, c)]
    if inflo:
        return inflo[0]
    
    ramif = [c for c in g.children(vid) if (g.edge_type(c) =='+')]
    if not ramif:
        return None
    if len(ramif) > 1: 
        print vid, genotype(vid), 'line ', g.node(vid)._line, len(ramif)
    return ramif[0]


for trunk in trunks:
    seq = []
    for vid in trunk:
        vars = []
        # Extract the next vertex that is the root of a sub-tree
        bid = filter_hamp_branch_crown(g, vid)
        if bid is None:
            vars = [-1]
        else:
            vars = [branching_type(bid, g)]

        inflo = is_inflo(g, bid) 
        branch_id = branching(g, vid)
        for name in variables:
            if inflo and name in inflo_variables:

                f = inflo_variables[name]
                vars.append(f(g.complex(bid)))
                print 'INFLO ', f(g.complex(bid))
            elif branch_id is not None: 
                f = variables[name]
                vars.append(f(branch_id))
            else:
                vars.append(0)

            #vars.extend([f(branching(g, vid)) if branching(g, vid) is not None elif inflo_variables[name] if is_inflo(g, vid) else 0 for f in variables.values()])
        vars.extend([f(vid) for f in global_variables.values()])
        seq.append(vars)
    final_seqs.append(seq)

#[[f(branching(g, vid)) if branching(g, vid) is not None else -1 for f in variables.values()] for vid in trunk] for trunk in trunks]
#final_seqs                          

In [ ]:
variables
variable_names = ['branching_type']+variables.keys()+global_variables.keys()
variable_names

In [ ]:
sequence_txt = analysis.write_sequences(final_seqs, variable_names, vertex_ids)
print sequence_txt

In [ ]:
f = open('strawberry_archi_node_trunk_path.seq', 'w') #seq
f.write(sequence_txt)
f.close()

In [ ]:
g.properties()['Crown_type']= type_of_crown(3)

In [ ]:
g.property('Crown_type')

In [ ]:
crown_type =type_of_crown(32032)
crown_type

In [ ]:
for t in trunks:
    print set(g.scale(v) for v in t )

In [ ]:
g.display(g)

In [ ]:
g[37632]

In [ ]:
t=list(apparent_axis(g,g.complex(37632)))

In [ ]:
set(is_inflo(g,v) for v in trunk)

In [ ]:
vtxs = list(c for t in trunks for v in t for c in g.children(v) if is_inflo(g, c))

In [ ]:
v=vtxs[0]
g.edge_type(v)
g[g.parent(v)]
g.parent(v)
is_inflo(g,filter_hamp_branch_crown(g, 826))
print g.children(826)
print g[838]

vid = 826
inflo = [c for c in g.children(vid) if is_inflo(g, c)]
print v